# ALLY Quickstart

In this notebook, we are going to run through some of the common tasks for creating data labeling agents with ALLY. In this example, we're going to create a data labeling agent for a text classification task - labeling our text samples as either "Subjective or "Objective" statements. 

This agent will be LLM-based, so we will use [OpenAI's API](https://platform.openai.com/). You will to generate an API key and set it as an environment variable as follows: 


Now, let's begin. 

## Dataset Creation
First, let's use a dataset of product reviews stored in pandas dataframe. This will help us manage our data as we add more attributes, like predictions and labels for subjectivity and objectivity over time. 

In [ ]:
import pandas as pd

df = pd.DataFrame([
  ["The mic is great.", "Subjective"],
  ["Will order from them again!", "Subjective"],
  ["Not loud enough and doesn't turn on like it should.", "Objective"],
  ["The phone doesn't seem to accept anything except CBR mp3s", "Objective"],
  ["All three broke within two months of use.", "Objective"]
], columns=["text", "ground_truth"])

df

We instantiate the environment

In [ ]:
import sys
sys.path.append('../')

from ally.environments.base import StaticEnvironment


env = environment=StaticEnvironment(
  df=df,
  ground_truth_columns={'predictions': 'ground_truth'}
)

## Create Agent

To create Agent, we need to to define 2 things:

**Skills** - Agent's abilities are defined as _Skills_. Each agent can possess many different skills. In our case, this agent only has one labeling skill, to produce a classification of Subjective or Objective for a given piece of text.  To define this skill, we will leverage an LLM, passing it instructions and the set of labeles we expect to receive back. 

**Environment** - that is where the Agent receives ground truth signal to improve its skill. Since we already created ground truth dataset, we can simply refer to the column from the dataframe. In the real world scenario, you may consider using a different environment where ground truth signal can be obtained asynchoronously by gathering real human feedback during agent's learning phase.

In [ ]:
from rich import print

from ally.agents.base import Agent
from ally.runtimes.openai import OpenAIRuntime
from ally.skills.collection.classification import ClassificationSkill
from app.core.settings import settings


agent = Agent(
    # define the agent's labeling skill that should classify text onto 2 categories
    skills=ClassificationSkill(
      name='subjectivity_detection',
      description='Understanding subjective and objective statements from text.',
      instruction_template='Classify a product review as either expressing "Subjective" or "Objective" statements.',
      input_template='Review: {text}',
    ),
    
    # basic environment extracts ground truth signal from the input records
    environment= env,
    
    runtimes = {
      # You can specify your OPENAI API KEY here via `OpenAIRuntime(..., api_key='your-api-key')`
      'openai': OpenAIRuntime(
        verbose=True,
        api_key=settings.openai_api_key,
        gpt_model_name="gpt-3.5-turbo",
        max_tokens=256,
      )
    },
    default_runtime='openai',
    
    teacher_runtimes = {
      'openai-gpt3': OpenAIRuntime(
        verbose=True,
        api_key=settings.openai_api_key,
        gpt_model_name="gpt-3.5-turbo",
        max_tokens=256,
        ),
      'openai-gpt4': OpenAIRuntime(
          verbose=True,
          api_key=settings.openai_api_key,
          gpt_model_name="gpt-4",
          max_tokens=256,
        )
    },
    
    # NOTE! If you don't have an access to gpt4 - replace it with "openai-gpt3"
    default_teacher_runtime='openai-gpt3'
)

print(agent)

## Learning Agent

We will now let Agent learn from the ground truth. After every action, Agent returns its _Experience_, where it stores various observations like predicted data, errors, accuracy, etc.

In [ ]:
agent.learn(learning_iterations=3, accuracy_threshold=0.95)

Let's see the final instructions:

In [ ]:
print(agent.skills)

... and predictions created by the skill:

In [ ]:
agent.run(dataset)

## Applying learned skills to the real data

Now as we have our Agent with evolved "subjectivity detection" skill, we can apply it to the real dataset without ground truth data:

In [ ]:
test_df = pd.DataFrame([
    "Doesn't hold charge.",
    "Excellent bluetooth headset",
    "I love this thing!",
    "VERY DISAPPOINTED."
], columns=['text'])
test_df

In [ ]:
predictions = agent.run(test_df)

In [ ]:
predictions